### Gold Layer - Business Aggregations

### Purpose
The Gold layer provides business-ready aggregated data optimized for reporting and dashboards.

### Metrics Created
- Total transactions per product category
- Total revenue
- Average order value

### Why Gold Layer?
- Designed for analytics and BI tools
- Simplifies reporting queries
- Provides a single source of truth for business KPIs

### Execution
- Reads cleaned data from Silver layer
- Writes aggregated results to Gold Delta table
- Runs automatically as part of a scheduled workflow


In [0]:
# Widgets (parameters)
dbutils.widgets.text("silver_table", "ecom_silver.transactions_silver", "silver_table")
dbutils.widgets.text("gold_table", "ecom_gold.transactions_gold", "gold_table")

silver_table = dbutils.widgets.get("silver_table")
gold_table = dbutils.widgets.get("gold_table")

In [0]:
from pyspark.sql import functions as F

silver = spark.table(silver_table)

In [0]:
gold_df = (
    silver
    .groupBy("Product_Category")
    .agg(
        F.count("*").alias("total_transactions"),
        F.sum("Purchase_Amount").alias("total_revenue"),
        F.avg("Purchase_Amount").alias("avg_order_value")
    )
    .orderBy(F.desc("total_revenue"))
)

In [0]:
gold_df.write.format("delta").mode("overwrite").saveAsTable(gold_table)


In [0]:
gold = spark.table(gold_table)
print("Gold rows:", gold.count())
display(gold.limit(10))


Gold rows: 8


Product_Category,total_transactions,total_revenue,avg_order_value
Sports,6312,3195335.9000000064,506.2319233206601
Toys,6392,3185652.360000008,498.3811576971226
Books,6253,3181897.3000000017,508.8593155285466
Clothing,6224,3171225.9600000037,509.5157390745507
Electronics,6320,3133965.039999991,495.88054430379606
Grocery,6215,3123579.520000008,502.58721158487657
Home & Kitchen,6209,3108945.780000005,500.716021903689
Beauty,6075,3057387.7899999907,503.2737102880643
